### Convert XML FILE TO AVRO

we are using hadoop 2.7.3 so Download avro-1.7.4.jar and avro-mapred-1.7.4-hadoop2.jar from apache server.

Step 1 : Build the jar from the maven project and upload it to azure cloud

In [ ]:
#execute these commands from root directory of your maven project will generate a JAR* file which will be used later
# Wordcount-1.0-SNAPSHOT.jar

mvn clean

mvn package -Dmaven.test.skip=true

In [ ]:
# export the jar file path 
# execute below command from the same directory you save jars

export LIBJARS=avro-1.7.7.jar,avro-mapred-1.7.7-hadoop2.jar
export HADOOP_CLASSPATH=avro-1.7.7.jar:avro-mapred-1.7.7-hadoop2.jar

execute the generated mvn jar file by passing path to xml file and path to output folder

In [ ]:
hadoop jar WordCount-1.0-SNAPSHOT.jar com.amal.stackOverflow.xmlToAvro -libjars $LIBJARS /LDZ/data/Posts.xml /LDZ/avro

at the end of the execution it will generate multiple avro files from the xml file which will be loaded to hive table later.

### create a hive table and load the avro data to hive db

In [ ]:
drop table if exists posts;
create external table posts(
  id            string,
  posttypeid    string,
  acceptedanswerid   string,
  parentid   string,
  creationdate string,
  score        string,
  viewcount    string,
  body         string,
  owneruserid  string,
  ownerdisplayname string,
  lasteditoruserid  string,
  lasteditordisplayname string,
  lasteditdate     string,
  lastactivitydate string,
  title            string,
  tags             string,
  answercount      string,
  commentcount     string,
  favoritecount    string,
  closeddate       string,
  communityowneddate string
)
stored as avro location '/LDZ/avro';

### check if data is loaded properly

In [ ]:
select count(*) from posts;

select id, to_date(creationdate) from posts limit 10;

### check count of unique dates and post types

In [ ]:
select count(distinct to_date(creationdate)) FROM posts;

Output : 
    
OK
3959
Time taken: 129.413 seconds, Fetched: 1 row(s)

In [ ]:
select count(distinct posttypeid) FROM posts;

Output:

OK
8
Time taken: 114.15 seconds, Fetched: 1 row(s)


### save unique dates to hdfs for further processing

In [ ]:
INSERT OVERWRITE DIRECTORY '/LDZ/temp/dates' SELECT distinct to_date(creationdate) FROM posts;

### create a bash script to save the dates from hdfs to local file system as a text file

In [ ]:
### we have 504 parts of files in hdfs starting from 0
### save each piece of data into a text file by appending the file
i="000000"
p="_0"
for i in $(seq -f "%06g" 0 504)
do
  hadoop fs -cat /LDZ/temp/dates/$i$p >> dates.txt
done

### BUILD DATAWAREHOUSE

### script to generate data warehouse with post dates and post types

since we have 8 unique post types and 3959 unique dates in dates.txt file. run the below script by passing the dates.txt file

export hive query result to a temp folder and then copy that data to the warehouse directory.

In [ ]:
#!/bin/bash
i="1"
while IFS= read -r line; do
    echo "Text read from file: '$line'"
    while [ $i -lt 9 ]
        do
        echo $i
        hive -e "INSERT OVERWRITE DIRECTORY '/LDZ/temp/day' STORED AS TEXTFILE select * from stackoverflow.posts WHERE to_date(creationdate)='$line' and posttypeid=='$i';"
        hadoop fs -mkdir -p /DWZ/$line/$i
        hadoop fs -cp /LDZ/temp/day /DWZ/$line/$i
        i=$[$i+1]
        done
    i="1"
done < "$1"


### QUESTIONS

1 - What are the top 10 most answered questions in Stack Overflow posts for a particular creation date?


In [ ]:
SELECT to_date(creationdate) as CreationDate, posttypeid, answercount, title from posts where to_date(creationdate)='2009-03-27' and posttypeid=='1' and answercount >=0 order by 1,2,3, answercount desc limit 10;

In [ ]:
hive> SELECT to_date(creationdate) as CreationDate, posttypeid, answercount, title from posts \
      where to_date(creationdate)='2009-03-27' and posttypeid=='1' and answercount >=0  \
      order by 1,2,3, answercount desc limit 10;


Task Execution Summary
------------------------------------------------------------------------------------------------------------------
  VERTICES  TOTAL_TASKS  FAILED_ATTEMPTS  KILLED_TASKS   DURATION(ms)  CPU_TIME(ms)  GC_TIME(ms)  INPUT_RECORDS  OUTPUT_RECORDS
------------------------------------------------------------------------------------------------------------------
     Map 1          136                0             0      116106.00     2,995,390      335,423     44,945,353              20
 Reducer 2            1                0             0       22223.00         2,410           20             20               0
------------------------------------------------------------------------------------------------------------------

OK
2009-03-27	1	9	What is the best way to learn Touch Typing?
2009-03-27	1	9	Build and Version Numbering for Java Projects (ant, cvs, hudson)
2009-03-27	1	9	Using app.config with a class library
2009-03-27	1	9	Why cast unused return values to void?
2009-03-27	1	9	Why do I need to know how many tests I will be running with Test::More?
2009-03-27	1	9	CSS, centered div, shrink to fit?
2009-03-27	1	9	Debugging scripts added via jQuery getScript function
2009-03-27	1	9	push_back for vector, deque and lists
2009-03-27	1	9	Does anyone use Right Outer Joins?
2009-03-27	1	8	Using SVG in GWT
Time taken: 120.106 seconds, Fetched: 10 row(s)
hive> 

## lets do it by script and generate a text file out of it


In [3]:
## output will be saved to popular.txt

In [ ]:
echo Hello, Which day you want to search for ? please enter the date in yyyy-mm-dd format.

read date

echo Which post type you want to look for ? please enter from 1 - 8.

read type

hive -e "SELECT to_date(creationdate) as CreationDate, posttypeid, answercount, title from posts where to_date(creationdate)='$date' and posttypeid=='$type' and answercount >=0 order by 1,2,3, answercount desc limit 10;" >> popular.txt


Question 2 - What’s the percentage of Stack Overflow questions that went unanswered in say 2015?


In [ ]:
SELECT year(to_date(creationdate)) as Creation_Date, COUNT(*) as total, SUM(CASE WHEN answercount = 0 THEN 1 ELSE 0 END) AS Unanswered, AVG(CASE WHEN answercount = 0 THEN 1 ELSE 0 END) * 100 AS Percentage FROM stackoverflow.posts where year(to_date(creationdate))='2015' GROUP BY year(to_date(creationdate)) ORDER BY Unanswered DESC;

OK
2015	5382945	728662	13.53649349937627
Time taken: 132.013 seconds, Fetched: 1 row(s)

In [ ]:
echo Hello, Which year you want to search ? 

read date
echo "CreationDate,TotalPosts,Percentage" >> question2.txt
hive -e "SELECT year(to_date(creationdate)) as Creation_Date, COUNT(*) as total, SUM(CASE WHEN answercount = 0 THEN 1 ELSE 0 END) AS Unanswered, AVG(CASE WHEN answercount = 0 THEN 1 ELSE 0 END) * 100 AS Percentage FROM stackoverflow.posts where year(to_date(creationdate))='$date' GROUP BY year(to_date(creationdate)) ORDER BY Unanswered DESC;" >> question2.txt


In [ ]:
******************** DONE *******************************